In [1]:
!pip install -U transformers trl peft datasets evaluate rouge_score underthesea bitsandbytes thefuzz bert_score sentence-transformers
# Tải tài nguyên NLTK cho METEOR
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# Bây giờ mới import các thư viện
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch

# ... (phần còn lại của hàm load_model_and_tokenizer và code gọi hàm)

/opt/jupyter_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import os
# os.environ["WANDB_DISABLED"] = "true"
import json
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import string
from underthesea import word_tokenize
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import evaluate

In [5]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch

def load_model_and_tokenizer(model_name, quantization="int8"):
    # Cấu hình quantization với bitsandbytes
    if quantization == "int8":
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,  # Quantize thành INT8
            bnb_8bit_compute_dtype=torch.bfloat16,  # Dùng bfloat16 để tính toán
            bnb_8bit_use_double_quant=True,  # Double quantization để tăng hiệu quả
        )
    elif quantization == "int4":
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,  # Quantize thành INT4
            bnb_4bit_compute_dtype=torch.bfloat16,  # Dùng bfloat16 để tính toán
            bnb_4bit_use_double_quant=True,  # Double quantization
            bnb_4bit_quant_type="nf4",  # Dùng NF4 (Normalized Float 4-bit) để tối ưu
        )
    else:
        quantization_config = None  # Không quantize

    # Tải tokenizer và mô hình
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,  # Áp dụng quantization
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )

    # Cấu hình LoRA
    peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        lora_dropout=0.5,  # Tăng dropout để giảm overfitting
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )
    model = get_peft_model(model, peft_config)
    return model, tokenizer, peft_config

# Chọn mô hình để thử (thay đổi model_name để thử từng mô hình)
model_name = "SeaLLMs/SeaLLMs-v3-7B-Chat" 
model, tokenizer, peft_config = load_model_and_tokenizer(model_name, quantization="int8")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.89s/it]


In [6]:
def formatting_func(example):
    if not all(k in example for k in ['input', 'output']):
        print('Thiếu key trong example:', example)
        return ''  # hoặc raise lỗi nếu bạn muốn dừng
    return f"<s>[INST] {example['input']} [/INST] {example['output']} </s>"

In [7]:
import os
import json
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import string
from underthesea import word_tokenize
from thefuzz import fuzz

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    text = " ".join(tokens)
    return text

def extract_json_from_folder(folder_path):
    dataset = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                    for item in json_data:
                        if all(k in item for k in ['input', 'output']):
                            item['input'] = preprocess_text(item['input'])
                            item['output'] = preprocess_text(item['output'])
                            dataset.append(item)
                        else:
                            print(f"Thiếu trường trong {filename}: {item}")
            except json.JSONDecodeError:
                print(f"Không thể parse JSON từ {filename}")
            except Exception as e:
                print(f"Lỗi khi đọc {filename}: {e}")
    return dataset

folder_path = "./data_finetune"
dataset = extract_json_from_folder(folder_path)

df = pd.DataFrame(dataset)
print("Số lượng giá trị duy nhất trong 'input' (exact):", df['input'].nunique())
print("Số lượng giá trị duy nhất trong 'output' (exact):", df['output'].nunique())
print("Tổng số hàng:", len(df))

# Fuzzy matching để tìm các record tương tự
similarity_threshold = 90  # Ngưỡng độ tương đồng (90%)
input_pairs = []
output_pairs = []

# Tìm các cặp input và output tương tự
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        input_sim = fuzz.ratio(df['input'].iloc[i], df['input'].iloc[j])
        if input_sim >= similarity_threshold:
            input_pairs.append((i, j, input_sim))
        output_sim = fuzz.ratio(df['output'].iloc[i], df['output'].iloc[j])
        if output_sim >= similarity_threshold:
            output_pairs.append((i, j, output_sim))

print(f"Số cặp input tương tự (>{similarity_threshold}%):", len(input_pairs))
print(f"Số cặp output tương tự (>{similarity_threshold}%):", len(output_pairs))

# Loại bỏ các record có input hoặc output tương tự (giữ record đầu tiên)
indices_to_keep = set(range(len(df)))
for i, j, _ in input_pairs:
    if j in indices_to_keep:
        indices_to_keep.remove(j)
for i, j, _ in output_pairs:
    if j in indices_to_keep:
        indices_to_keep.remove(j)

df = df.iloc[list(indices_to_keep)].reset_index(drop=True)
print("Số hàng sau khi xóa record tương tự:", len(df))

# Kiểm tra lại độ unique
print("Số lượng giá trị duy nhất trong 'input' (sau xử lý):", df['input'].nunique())
print("Số lượng giá trị duy nhất trong 'output' (sau xử lý):", df['output'].nunique())

# Chia train/validation
full_dataset = Dataset.from_pandas(df[['input', 'output']])
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df[['input', 'output']])
eval_dataset = Dataset.from_pandas(eval_df[['input', 'output']])
print(train_df[['input', 'output']].head())

Số lượng giá trị duy nhất trong 'input' (exact): 1599
Số lượng giá trị duy nhất trong 'output' (exact): 1659
Tổng số hàng: 1700
Số cặp input tương tự (>90%): 320
Số cặp output tương tự (>90%): 41
Số hàng sau khi xóa record tương tự: 1471
Số lượng giá trị duy nhất trong 'input' (sau xử lý): 1471
Số lượng giá trị duy nhất trong 'output' (sau xử lý): 1471
                                                  input  \
998   trẻ không biết tôn trọng lượt nói trong thảo l...   
254                             trẻ nói ít phải làm sao   
1073  trẻ không biết chuẩn bị nội dung khi báo cáo nhóm   
643   trẻ không biết nói lời chia tay khi bạn chuyển...   
1450  tự kỷ có học được kỹ năng kết thúc tương tác x...   

                                                 output  
998   nói chen hoặc ngắt lời bạn là thiếu kỹ năng xã...  
254   cần đánh giá xem trẻ có hiểu lời tương tác bằn...  
1073  nội dung rời rạc là thiếu chuẩn bị – nên luyện...  
643   không thể hiện chia tay là thiếu kỹ năng chia ...  


In [8]:
import evaluate
import numpy as np
from sentence_transformers import SentenceTransformer, util
def evaluate_metrics(predictions, references):
    # Tải các độ đo
    rouge = evaluate.load("rouge")
    meteor = evaluate.load("meteor")

    # Tính các độ đo
    rouge_results = rouge.compute(predictions=predictions, references=references)
    meteor_results = meteor.compute(predictions=predictions, references=references)

    # Tải mô hình nhúng câu để tính Cosine Similarity
    embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Tính embeddings cho dự đoán và tham chiếu
    pred_embeddings = embedder.encode(predictions, convert_to_tensor=True)
    ref_embeddings = embedder.encode(references, convert_to_tensor=True)

    # Tính Cosine Similarity giữa từng cặp dự đoán-tham chiếu
    cosine_scores = util.cos_sim(pred_embeddings, ref_embeddings)
    # Lấy trung bình Cosine Similarity (chỉ lấy đường chéo chính, vì mỗi dự đoán chỉ so với tham chiếu tương ứng)
    avg_cosine_similarity = np.mean([cosine_scores[i][i].item() for i in range(len(predictions))])
    
    # Gộp kết quả
    metrics = {
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "meteor": meteor_results["meteor"],
        "cosine_similarity": avg_cosine_similarity,
    }
    return metrics

def generate_predictions(model, tokenizer, inputs, max_length=200):
    """Tạo dự đoán từ mô hình cho các đầu vào."""
    model.eval()
    predictions = []
    # Đảm bảo pad_token được thiết lập
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    for input_text in inputs:
        # Tiền xử lý input để đồng bộ với huấn luyện
        input_text = preprocess_text(input_text)
        prompt = f"<s>[INST] {input_text} [/INST]"
        inputs_encoded = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs_encoded,
                max_length=max_length,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id
            )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred = generated_text.split("[/INST]")[-1].strip()
        # Tiền xử lý dự đoán để đồng bộ với tham chiếu
        pred = preprocess_text(pred)
        predictions.append(pred)
    return predictions

In [9]:
import json
import gc
import time
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import EarlyStoppingCallback

# Loại bỏ cột không cần thiết để tránh cảnh báo
train_dataset = train_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in train_dataset.column_names else [])
eval_dataset = eval_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in eval_dataset.column_names else [])

# Đo thời gian chạy đơn
print("\n=== Bắt đầu chạy đơn ===")
start_time = time.time()
# Cấu hình huấn luyện cho Single Run
training_arguments_single = TrainingArguments(
    output_dir="./results_single_seaLLM",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.1,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    eval_steps=10,
    logging_strategy="steps",
    log_level="info",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Huấn luyện Single Run
trainer_single = SFTTrainer(
    model=model,
    args=training_arguments_single,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01,
        )
    ],
)
print(f"Train dataset size: {len(train_dataset)}, Eval dataset size: {len(eval_dataset)}")
trainer_single.train()

# Lưu mô hình Single Run
model.save_pretrained("./finetuned_seaLLM_single")
tokenizer.save_pretrained("./finetuned_seaLLM_single")

# Đánh giá Single Run
test_inputs = eval_df['input'].tolist()
test_references = eval_df['output'].tolist()
predictions_single = generate_predictions(model, tokenizer, test_inputs)
metrics_single = evaluate_metrics(predictions_single, test_references)
print("Single Run Metrics:", metrics_single)

# Lưu metrics vào file
with open("single_run_metrics.json", "w") as f:
    json.dump(metrics_single, f, indent=4)

end_time = time.time()
duration = end_time - start_time
print(f"Thời gian chạy đơn: {duration:.2f} giây")
# Kiểm tra mẫu dự đoán
for i in range(5):
    print(f"Input: {test_inputs[i]}")
    print(f"Prediction: {predictions_single[i]}")
    print(f"Reference: {test_references[i]}\n")
del model, trainer_single
torch.cuda.empty_cache()
gc.collect()


=== Bắt đầu chạy đơn ===


Truncating eval dataset: 100%|██████████| 295/295 [00:00<00:00, 127901.56 examples/s]
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,176
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 219
  Number of trainable parameters = 80,740,352


Train dataset size: 1176, Eval dataset size: 295


/opt/jupyter_venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
10,3.368900,3.509594
20,3.108500,2.584057
30,2.221300,1.980858
40,1.861000,1.800578
50,1.711100,1.675380
60,1.643400,1.639721
70,1.572700,1.562272
80,1.469700,1.562473
90,1.446700,1.493969
100,1.365400,1.478668


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2

Single Run Metrics: {'rouge1': np.float64(0.6209654627664118), 'rouge2': np.float64(0.2679127253546244), 'rougeL': np.float64(0.3886934998629489), 'meteor': np.float64(0.2989020973699027), 'cosine_similarity': np.float64(0.7166761577129364)}
Thời gian chạy đơn: 2128.68 giây
Input: trẻ không biết tôn trọng ranh giới của người khác
Prediction: không biết giữ khoảng cách khi nói chuyện hoặc chạm vào người khác là thiếu kỹ năng xã hội và thể hiện thiếu tôn trọng cá nhân s
Reference: cố ép bạn làm theo ý mình là chưa có kỹ năng đồng thuận – nên luyện hỏi bạn có muốn chơi thế này không trước khi bắt đầu

Input: trẻ không dùng tay để chỉ vật
Prediction: trẻ không dùng tay để chỉ vật có thể là dấu hiệu chậm phát triển ngôn ngữ động tác hoặc khả năng xã hội – cần được đánh giá chuyên sâu s
Reference: việc trẻ không dùng tay để chỉ khi muốn thể hiện nhu cầu là dấu hiệu quan trọng để nhận biết trẻ có khó khăn trong giao tiếp phi ngôn ngữ

Input: tự kỷ và rối loạn ngôn ngữ khác gì nhau
Prediction:

40424

In [10]:
# Cell 8: 5-Fold Cross-Validation
from sklearn.model_selection import KFold
import numpy as np
import gc
import time


# Cấu hình KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_metrics = []
fold_models = []
fold_times = []
total_start_time = time.time()

# Lặp qua từng fold
for fold, (train_idx, eval_idx) in enumerate(kf.split(df)):
    print(f"\nTraining Fold {fold + 1}...")
    fold_start_time = time.time()
    
    # Tạo tập train và eval cho fold hiện tại
    train_fold = df.iloc[train_idx][['input', 'output']]
    eval_fold = df.iloc[eval_idx][['input', 'output']]
    train_fold_dataset = Dataset.from_pandas(train_fold)
    eval_fold_dataset = Dataset.from_pandas(eval_fold)

    # Loại bỏ cột không cần thiết
    train_fold_dataset = train_fold_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in train_fold_dataset.column_names else [])
    eval_fold_dataset = eval_fold_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in eval_fold_dataset.column_names else [])

    # Tải lại mô hình gốc với INT4 quantization
    model, tokenizer, peft_config = load_model_and_tokenizer(model_name,quantization="int4")
    print(f"Fold {fold + 1} - Train size: {len(train_fold_dataset)}, Eval size: {len(eval_fold_dataset)}")
    # Cấu hình huấn luyện cho fold
    training_arguments_fold = TrainingArguments(
        output_dir=f"./results_fold_{fold + 1}",
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        optim="paged_adamw_32bit",
        save_steps=100,
        logging_steps=10,
        learning_rate=5e-5,
        weight_decay=0.1,
        fp16=False,
        bf16=True,
        max_grad_norm=0.3,
        warmup_ratio=0.1,
        group_by_length=True,
        lr_scheduler_type="cosine",
        eval_strategy="steps",
        eval_steps=10,
        logging_strategy="steps",
        log_level="info",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

    # Huấn luyện fold
    trainer_fold = SFTTrainer(
        model=model,
        args=training_arguments_fold,
        train_dataset=train_fold_dataset,
        eval_dataset=eval_fold_dataset,
        peft_config=peft_config,
        formatting_func=formatting_func,
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=3,
                early_stopping_threshold=0.01,
            )
        ],
    )
    trainer_fold.train()

    # Lưu mô hình fold
    fold_path = f"./finetuned_seaLLM_fold_{fold + 1}"
    model.save_pretrained(fold_path)
    tokenizer.save_pretrained(fold_path)
    fold_models.append(fold_path)

    # Đánh giá fold
    test_inputs_fold = eval_fold['input'].tolist()
    test_references_fold = eval_fold['output'].tolist()
    predictions_fold = generate_predictions(model, tokenizer, test_inputs_fold)
    metrics_fold = evaluate_metrics(predictions_fold, test_references_fold)
    print(f"Fold {fold + 1} Metrics:", metrics_fold)
    fold_metrics.append(metrics_fold)

    # Lưu metrics của fold
    with open(f"fold_{fold + 1}_metrics.json", "w") as f:
        json.dump(metrics_fold, f, indent=4)

    fold_end_time = time.time()
    fold_duration = fold_end_time - fold_start_time
    fold_times.append(fold_duration)
    print(f"Thời gian chạy Fold {fold + 1}: {fold_duration:.2f} giây")
    
    # Dọn dẹp bộ nhớ
    del model, trainer_fold
    torch.cuda.empty_cache()
    gc.collect()

# Tính tổng thời gian và thời gian trung bình
total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"\n=== Kết thúc huấn luyện K-Fold ===")
print(f"Tổng thời gian chạy: {total_duration:.2f} giây")
print(f"Thời gian trung bình mỗi fold: {np.mean(fold_times):.2f} giây")

# Tính trung bình metrics qua các fold
avg_metrics = {
    "rouge1": np.mean([m["rouge1"] for m in fold_metrics]),
    "rouge2": np.mean([m["rouge2"] for m in fold_metrics]),
    "rougeL": np.mean([m["rougeL"] for m in fold_metrics]),
    "meteor": np.mean([m["meteor"] for m in fold_metrics]),
    "cosine_similarity": np.mean([m["cosine_similarity"] for m in fold_metrics]),
}
print("\nAverage Cross-Validation Metrics:", avg_metrics)

# Lưu metrics trung bình
with open("cross_validation_seaLLM_metrics.json", "w") as f:
    json.dump(avg_metrics, f, indent=4)

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a99


Training Fold 1...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 131072,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": false,
  "use_sliding_window": false,
  "vocab_siz

Fold 1 - Train size: 1176, Eval size: 295


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncating eval dataset: 100%|██████████| 295/295 [00:00<00:00, 133648.70 examples/s]
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,176
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimizatio

...

Fold 5 - Train size: 1177, Eval size: 294


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncating eval dataset: 100%|██████████| 294/294 [00:00<00:00, 121132.16 examples/s]
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,177
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimizatio

Step,Training Loss,Validation Loss
10,3.473000,3.527105
20,3.161300,2.558093
30,2.254800,1.979399
40,1.852100,1.800944
50,1.730000,1.695969
60,1.572000,1.626179
70,1.587200,1.578608
80,1.694900,1.542922
90,1.398600,1.501430
100,1.405500,1.472815


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 294
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 294
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, input, output. If text, input, output are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2

Fold 5 Metrics: {'rouge1': np.float64(0.6199962413803413), 'rouge2': np.float64(0.26565730440611623), 'rougeL': np.float64(0.37761908175709924), 'meteor': np.float64(0.29684600750577994), 'cosine_similarity': np.float64(0.7089269719511068)}
Thời gian chạy Fold 5: 1371.63 giây

=== Kết thúc huấn luyện K-Fold ===
Tổng thời gian chạy: 6784.18 giây
Thời gian trung bình mỗi fold: 1356.37 giây

Average Cross-Validation Metrics: {'rouge1': np.float64(0.6183817264864322), 'rouge2': np.float64(0.2664238453128201), 'rougeL': np.float64(0.3839010571763791), 'meteor': np.float64(0.29856785575136047), 'cosine_similarity': np.float64(0.7079354179408377)}


In [11]:
import json
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer#, BitsAndBytesConfig # Không cần BitsAndBytesConfig cho bước này
from peft import PeftModel
import gc

# --- Phần 1: Tìm fold có metric tốt nhất ---

def load_metrics_from_file(file_path):
    """Đọc metrics từ file JSON."""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Cảnh báo: Không tìm thấy file metric: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Cảnh báo: File metric không phải JSON hợp lệ: {file_path}")
        return None

def find_best_fold_by_metric(metrics_base_path, num_folds, metric_name_to_optimize, higher_is_better=True):
    """
    Tìm fold có giá trị metric được chỉ định cao nhất (hoặc thấp nhất).

    Args:
        metrics_base_path (str): Đường dẫn cơ sở đến thư mục chứa các file metrics.
                                 Ví dụ: "./" nếu các file ở thư mục hiện tại.
        num_folds (int): Tổng số lượng folds.
        metric_name_to_optimize (str): Tên của metric dùng để so sánh (ví dụ: 'rougeL', 'cosine_similarity').
        higher_is_better (bool): True nếu giá trị metric cao hơn là tốt hơn, False nếu thấp hơn là tốt hơn.

    Returns:
        tuple: (best_fold_number, best_metric_value, all_metrics_of_best_fold) hoặc (None, None, None) nếu lỗi.
    """
    best_fold_so_far = None
    best_metric_val = -float('inf') if higher_is_better else float('inf')
    all_metrics_best_fold = None

    print(f"Đang tìm fold tốt nhất dựa trên metric: '{metric_name_to_optimize}' (cao hơn là tốt hơn: {higher_is_better})")

    for i in range(1, num_folds + 1):
        # Giả sử tên file là "fold_X_metrics" hoặc "fold_X_metrics.json"
        # Sửa lại mẫu tên file nếu cần
        metric_file_candidate_1 = os.path.join(metrics_base_path, f"fold_{i}_metrics.json")
        metric_file_candidate_2 = os.path.join(metrics_base_path, f"fold_{i}_metrics") # Không có đuôi .json

        metrics_data = None
        if os.path.exists(metric_file_candidate_1):
            metrics_data = load_metrics_from_file(metric_file_candidate_1)
        elif os.path.exists(metric_file_candidate_2):
            metrics_data = load_metrics_from_file(metric_file_candidate_2)
        else:
            print(f"Không tìm thấy file metric cho fold {i} tại: {metric_file_candidate_1} hoặc {metric_file_candidate_2}")
            continue

        if metrics_data is None:
            continue # Bỏ qua nếu không đọc được file

        if metric_name_to_optimize not in metrics_data:
            print(f"Cảnh báo: Metric '{metric_name_to_optimize}' không có trong file của fold {i}. Bỏ qua fold này để so sánh.")
            continue

        current_metric_val = metrics_data[metric_name_to_optimize]
        print(f"Fold {i}: '{metric_name_to_optimize}' = {current_metric_val:.4f}")

        if higher_is_better:
            if current_metric_val > best_metric_val:
                best_metric_val = current_metric_val
                best_fold_so_far = i
                all_metrics_best_fold = metrics_data
        else: # lower is better
            if current_metric_val < best_metric_val:
                best_metric_val = current_metric_val
                best_fold_so_far = i
                all_metrics_best_fold = metrics_data

    if best_fold_so_far is not None:
        print(f"\n=> Fold tốt nhất được chọn: Fold {best_fold_so_far} với {metric_name_to_optimize} = {best_metric_val:.4f}")
        # print(f"Toàn bộ metrics của fold {best_fold_so_far}: {all_metrics_best_fold}")
        return best_fold_so_far, best_metric_val, all_metrics_best_fold
    else:
        print(f"\n=> Không thể xác định fold tốt nhất dựa trên metric '{metric_name_to_optimize}'.")
        return None, None, None

# --- Cấu hình cho việc tìm fold ---
# Đặt đường dẫn đến thư mục chứa các file fold_X_metrics của bạn
# Ví dụ: nếu các file fold_1_metrics, fold_2_metrics,... nằm cùng thư mục với script này:
METRICS_FILES_DIRECTORY = "./" 
NUM_TOTAL_FOLDS = 5
# Chọn metric bạn muốn sử dụng để quyết định fold nào tốt nhất
# Ví dụ: 'rougeL', 'cosine_similarity', 'meteor', 'rouge1', 'rouge2'
METRIC_TO_OPTIMIZE_FOR = "cosine_similarity" # THAY ĐỔI TÊN METRIC NÀY NẾU CẦN

best_fold_id, _, _ = find_best_fold_by_metric(
    METRICS_FILES_DIRECTORY,
    NUM_TOTAL_FOLDS,
    METRIC_TO_OPTIMIZE_FOR,
    higher_is_better=True # Hầu hết các metric này, cao hơn là tốt hơn
)

if best_fold_id is None:
    print("Không thể xác định fold tốt nhất. Sẽ sử dụng một fold mặc định hoặc dừng chương trình.")
    # Gán một fold mặc định nếu muốn tiếp tục
    default_fold_if_not_found = 4 # Ví dụ, bạn có thể muốn mặc định là fold 4
    print(f"Sử dụng fold mặc định: {default_fold_if_not_found}")
    best_fold_id = default_fold_if_not_found
    # Hoặc bạn có thể dừng chương trình:
    # exit("Dừng chương trình do không tìm được fold tốt nhất.")

# --- Phần 2: Merge model sử dụng adapter từ fold tốt nhất ---
# Sửa đổi để merge vào base model ở định dạng full/half precision

print(f"\n--- Bắt đầu quá trình merge model cho Fold {best_fold_id} ---")

try:
    # Giải phóng bộ nhớ trước khi tải model lớn
    torch.cuda.empty_cache()
    gc.collect()
    print("Đã giải phóng bộ nhớ GPU (nếu có).")

    # Cấu hình tải mô hình gốc
    base_model_name = "SeaLLMs/SeaLLMs-v3-7B-Chat" # Thay bằng model base của bạn nếu khác
    # Dựa trên lỗi trước đó, model Vinallama có tên khác (Viet-Mistral/Vinallama-7B-Chat?)
    # Bạn cần chắc chắn base_model_name ở đây là model gốc bạn đã dùng để fine-tune
    # Ví dụ: base_model_name = "Viet-Mistral/Vinallama-7B-Chat" # <-- KHẢ NĂNG CAO BẠN CẦN THAY ĐỔI Ở ĐÂY

    print(f"Đang tải tokenizer cho model: {base_model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    print("Tokenizer đã được tải.")

    # Tải base model ở định dạng Bfloat16 (hoặc Float16) - KHÔNG DÙNG BitsAndBytes
    print(f"Đang tải base model '{base_model_name}' ở định dạng BF16 (không lượng tử hóa BitsAndBytes)...")
    # Bỏ hoàn toàn quantization_config khi tải base model
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        # quantization_config=quantization_config, # <-- BỎ DÒNG NÀY!
        torch_dtype=torch.bfloat16, # Tải ở bfloat16 (kích thước lớn)
        device_map="cuda" # Vẫn dùng auto device_map để phân bổ lên GPU nếu có đủ VRAM
    )
    print("Base model đã được tải thành công ở định dạng BF16.")

    # Đường dẫn tới adapter LoRA của fold tốt nhất
    # Cần điều chỉnh đường dẫn này cho phù hợp với cấu trúc thư mục của bạn
    # Đảm bảo bạn sử dụng tên thư mục fine-tuned adapter đúng với fold tốt nhất tìm được ở Phần 1
    fine_tuned_adapters_base_dir = "/home/thanhnguyenvq2403/model/KLTN/" # Giả định thư mục chứa fine-tuned models
    adapter_path_for_best_fold = os.path.join(fine_tuned_adapters_base_dir, f"finetuned_seaLLM_fold_{best_fold_id}") # <--- SỬA TÊN THƯ MỤC ADAPTER NẾU CẦN (ví dụ: vinallama thay vì seaLLM)

    print(f"Đang tải adapter LoRA từ: {adapter_path_for_best_fold}")
    if not os.path.isdir(adapter_path_for_best_fold):
        raise FileNotFoundError(f"Lỗi: Thư mục adapter LoRA không tồn tại: {adapter_path_for_best_fold}")

    lora_model = PeftModel.from_pretrained(base_model, adapter_path_for_best_fold, is_trainable=False)
    print("Adapter LoRA đã được tải.")

    # Hợp nhất adapter vào base model (ở định dạng BF16)
    # Kết quả merged_model sẽ là mô hình dense ở định dạng BF16
    print("Đang hợp nhất adapter LoRA vào base model (BF16)...")
    merged_model = lora_model.merge_and_unload()
    print("Hợp nhất adapter thành công. Mô hình đã hợp nhất ở định dạng BF16.")

    # Lưu mô hình đã hợp nhất
    # Đặt tên cho thư mục lưu model đã merge.
    # Tên này nên phản ánh rằng nó đã merge và ở định dạng không lượng tử hóa BitsAndBytes
    output_merged_model_dir = f"/home/thanhnguyenvq2403/model/KLTN/merged_seaLLM_fold_{best_fold_id}_bf16" # Đổi tên cho rõ định dạng
    print(f"Đang lưu mô hình đã hợp nhất vào: {output_merged_model_dir}")

    # Đảm bảo thư mục output tồn tại
    os.makedirs(output_merged_model_dir, exist_ok=True)

    merged_model.save_pretrained(output_merged_model_dir, safe_serialization=True) # Nên dùng safe_serialization
    tokenizer.save_pretrained(output_merged_model_dir) # Lưu cả tokenizer

    print(f"Mô hình đã hợp nhất và tokenizer đã được lưu vào: {output_merged_model_dir}")

    # Dọn dẹp bộ nhớ
    del base_model
    del lora_model
    del merged_model
    torch.cuda.empty_cache()
    gc.collect()
    print("Đã dọn dẹp bộ nhớ.")


except FileNotFoundError as e:
     print(f"LỖI FILE: {e}")
except Exception as e:
    print(f"ĐÃ CÓ LỖI XẢY RA TRONG QUÁ TRÌNH MERGE MODEL: {e}")
    import traceback
    traceback.print_exc()

Đang tìm fold tốt nhất dựa trên metric: 'cosine_similarity' (cao hơn là tốt hơn: True)
Fold 1: 'cosine_similarity' = 0.7108
Fold 2: 'cosine_similarity' = 0.7008
Fold 3: 'cosine_similarity' = 0.7117
Fold 4: 'cosine_similarity' = 0.7075
Fold 5: 'cosine_similarity' = 0.7089

=> Fold tốt nhất được chọn: Fold 3 với cosine_similarity = 0.7117

--- Bắt đầu quá trình merge model cho Fold 3 ---
Đã giải phóng bộ nhớ GPU (nếu có).

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a99


Đang tải tokenizer cho model: SeaLLMs/SeaLLMs-v3-7B-Chat...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer đã được tải.
Đang tải base model 'SeaLLMs/SeaLLMs-v3-7B-Chat' ở định dạng BF16 (không lượng tử hóa BitsAndBytes)...


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-7B-Chat/snapshots/a9900348910a6d7f611a6859270d6c28da1e0789/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 131072,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": false,
  "use_sliding_window": false,
  "vocab_size": 152064
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--SeaLLMs--SeaLL

Base model đã được tải thành công ở định dạng BF16.
Đang tải adapter LoRA từ: /home/thanhnguyenvq2403/model/KLTN/finetuned_seaLLM_fold_3
LỖI FILE: Lỗi: Thư mục adapter LoRA không tồn tại: /home/thanhnguyenvq2403/model/KLTN/finetuned_seaLLM_fold_3
